# Bayesian Hyper Parameter Tuning with Optuna

Bayesian Optimization is a method used for optimizing expensive-to-evaluate functions, particularly useful in hyperparameter tuning for machine learning models. Here's a clear explanation of how it works and the math behind it:

Blog post: [Bayesian optimization for hyper parameter tuning](https://www.machinelearningplus.com/machine-learning/bayesian-optimization-for-hyperparameter-tuning/)

## Overview of Bayesian Optimization

__1. Surrogate Model:__ Bayesian optimization builds a probabilistic model of the objective function (often using Gaussian processes or other regression models).

__2. Acquisition Function:__ It uses this model to guide the search for the next set of hyperparameters to evaluate by optimizing an acquisition function.

__3. Evaluate and Update:__ The chosen hyperparameters are evaluated using the true objective function (e.g., model performance on validation data). The surrogate model is then updated with this new information.

__4. Iterate:__ Steps 2 and 3 are repeated until a stopping criterion is met (e.g., a fixed number of iterations or convergence).

In [2]:
# pip install optuna scikit-learn

## Import packages and dataset

In [3]:
import optuna
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Define the objective function

In [4]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }
    
    model = RandomForestClassifier(random_state=42, **params)
    model.fit(X_train, y_train)
    val_preds = model.predict(X_val)
    accuracy = accuracy_score(y_val, val_preds)
    return accuracy


## Run Optuna Bayesian Optimization

In [5]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Number of finished trials:", len(study.trials))
print("Best trial:", study.best_trial.params)
print("Best accuracy:", study.best_trial.value)

[I 2024-08-02 07:34:06,703] A new study created in memory with name: no-name-464d5fb0-21a8-40d2-8925-15deccd4f719
[I 2024-08-02 07:34:08,625] Trial 0 finished with value: 0.9649122807017544 and parameters: {'n_estimators': 800, 'max_depth': 7, 'min_samples_split': 11, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.9649122807017544.
[I 2024-08-02 07:34:09,581] Trial 1 finished with value: 0.9649122807017544 and parameters: {'n_estimators': 500, 'max_depth': 15, 'min_samples_split': 13, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 0 with value: 0.9649122807017544.
[I 2024-08-02 07:34:10,145] Trial 2 finished with value: 0.9649122807017544 and parameters: {'n_estimators': 300, 'max_depth': 11, 'min_samples_split': 15, 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 0 with value: 0.9649122807017544.
[I 2024-08-02 07:34:12,117] Trial 3 finished with value: 0.9649122807017544 and parameters: {'n_estimators': 1000, 'max_depth': 9,

Number of finished trials: 100
Best trial: {'n_estimators': 800, 'max_depth': 7, 'min_samples_split': 11, 'min_samples_leaf': 7, 'max_features': 'sqrt'}
Best accuracy: 0.9649122807017544


## Train model 

In [6]:
best_params = study.best_trial.params
final_model = RandomForestClassifier(random_state=42, **best_params)
final_model.fit(X_train, y_train)
test_preds = final_model.predict(X_val)
test_accuracy = accuracy_score(y_val, test_preds)

print("Test Accuracy with best hyperparameters:", test_accuracy)

Test Accuracy with best hyperparameters: 0.9649122807017544


---


## Using GridSearchCV

Exhaustively searches over a manually specified parameter grid, making it suitable for exploring a relatively small search space but can be computationally expensive.

In [7]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'max_depth': [3, 5, 7, 10, 15],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': ['sqrt', 'log2']
}

# Create a base RandomForestClassifier
base_model = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid,
                           scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Print results
print("Best parameters found by GridSearchCV:")
print(grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Evaluate on validation set
best_model = grid_search.best_estimator_
val_preds = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_preds)
print("Validation accuracy with best hyperparameters:", val_accuracy)


Fitting 5 folds for each of 800 candidates, totalling 4000 fits
Best parameters found by GridSearchCV:
{'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best cross-validation accuracy: 0.9626373626373625
Validation accuracy with best hyperparameters: 0.9649122807017544


## Random Search

`RandomizedSearchCV` works by sampling a fixed number of hyperparameter settings from the specified distributions. 

It is more efficient for larger search spaces and can be particularly useful when computational resources are limited or when searching over a very large number of parameters

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define the parameter distributions for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 15),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2']
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=base_model, param_distributions=param_dist,
                                   n_iter=100, scoring='accuracy', cv=5, verbose=1, n_jobs=-1, random_state=42)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Print results
print("Best parameters found by RandomizedSearchCV:")
print(random_search.best_params_)
print("Best cross-validation accuracy:", random_search.best_score_)

# Evaluate on validation set
best_random_model = random_search.best_estimator_
val_preds_random = best_random_model.predict(X_val)
val_accuracy_random = accuracy_score(y_val, val_preds_random)
print("Validation accuracy with best hyperparameters (RandomizedSearchCV):", val_accuracy_random)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters found by RandomizedSearchCV:
{'max_depth': 11, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 996}
Best cross-validation accuracy: 0.9582417582417582
Validation accuracy with best hyperparameters (RandomizedSearchCV): 0.9649122807017544


---

<br><br><br><br>
## Using GPyOpt

In [10]:
# pip install gpyopt

In [11]:
import numpy as np
import GPy
import GPyOpt
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [12]:
# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

In [13]:
# Define the objective function
def objective_function(parameters):
    # Convert the hyperparameters to integers
    parameters = parameters[0]
    n_estimators = int(parameters[0])
    max_depth = int(parameters[1])
    
    # Define the classifier with the specified hyperparameters
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=0)
    
    # Calculate cross-validation scores (accuracy)
    scores = cross_val_score(clf, X, y, cv=5)
    
    # Return the average accuracy (to be maximized)
    return -np.mean(scores)  # Minimize negative accuracy -> maximize accuracy

# Bounds for hyperparameters (n_estimators and max_depth)
bounds = [
    {'name': 'n_estimators', 'type': 'discrete', 'domain': (10, 50, 100, 150)},
    {'name': 'max_depth', 'type': 'discrete', 'domain': (2, 5, 10, 20)},
]

# Initialize Bayesian Optimization
optimizer = GPyOpt.methods.BayesianOptimization(f=objective_function, 
                                                domain=bounds, 
                                                acquisition_type='EI')

# Run the optimization
optimizer.run_optimization(max_iter=10)  # Specify the number of iterations

# Get the best hyperparameters found
best_params = optimizer.X[np.argmin(optimizer.Y)]

# Convert to integer values
best_n_estimators = int(best_params[0])
best_max_depth = int(best_params[1])

print("Best n_estimators:", best_n_estimators)
print("Best max_depth:", best_max_depth)
print("Best accuracy found:", -optimizer.fx_opt)  # Convert back from negative accuracy

Best n_estimators: 100
Best max_depth: 20
Best accuracy found: 0.9666666666666668


__Rebuild the best model with the chosen params__

In [ ]:
# Train the final model with the best hyperparameters
best_model = RandomForestClassifier(n_estimators=best_n_estimators, max_depth=best_max_depth, random_state=0)
best_model.fit(X, y)

# Split the data into training and testing sets for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model on the training set
best_model.fit(X_train, y_train)

# Predict on the testing set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy of the best model:", accuracy)

## Simple example of surrogate function and acquisition function

In [18]:
import numpy as np
import GPy
import GPyOpt

# Sample data points (example)
X = np.array([[1.0], [3.0], [5.0], [6.0], [8.0]])
Y = np.sin(X)  # Example function to approximate

# Define the Gaussian Process model
kernel = GPy.kern.RBF(input_dim=1, variance=1.0, lengthscale=1.0)
gp_model = GPy.models.GPRegression(X, Y, kernel)

# Optimize (find the maximum likelihood parameters)
gp_model.optimize()

# Predict mean and variance at new points
X_new = np.array([[2.0], [4.0], [7.0]])
mean, var = gp_model.predict(X_new)

print("Predicted mean:")
print(mean)

print("Predicted variance:")
print(var)

# Define the bounds and domain for optimization (example)
bounds = [{'name': 'x', 'type': 'continuous', 'domain': (0, 10)}]

# Define the acquisition function (Expected Improvement)
acquisition_function = GPyOpt.acquisitions.AcquisitionEI(gp_model, space=bounds)

# Get the next point to evaluate
next_point = acquisition_function.acq_max(bounds=bounds, opt_model=gp_model)

print("Next point to evaluate:")
print(next_point)

Predicted mean:
[[ 0.75670972]
 [-0.70565835]
 [ 0.6263472 ]]
Predicted variance:
[[0.03747988]
 [0.01773146]
 [0.02257624]]
